In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/[2023-2024] AN2DL/HW1/

Mounted at /gdrive
/gdrive/My Drive/[2023-2024] AN2DL/HW1/LAST


#Import Libraries

In [2]:
%pip install keras-cv
%pip install seaborn

#Import general libraries

import numpy as np
from numpy import load
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


#Import keras and tf

import keras_cv
import tensorflow as tf
from tensorflow import keras as tfk
from keras import layers as tfkl
import logging
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation
from keras.preprocessing.image import ImageDataGenerator

#Fix seeds for reproducibility
seed = 42
np.random.seed = seed

#Some settings
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 46.2 MB/s eta 0:00:00
Using TensorFlow backend
2.14.0


#Load Dataset

In [4]:
#Load the set
dataset = load("public_data.npz", allow_pickle=True)
lst = dataset.files
for name in lst:
    array = dataset[name]  # Access the array by its name
    # You can perform operations on the array here
    print(f"Array Name: {name}")
    print("Array Shape:")
    print(array.shape)

#Calculate numeric labels
unique_labels = np.unique(dataset['labels'])
label_to_int = {label: index for index, label in enumerate(unique_labels)}
label_to_int
desired_output = [label_to_int[lab] for lab in dataset['labels']]

LabelDataFrame = pd.DataFrame(dataset['labels'])
intDataFrame = pd.DataFrame(desired_output)

print(f"Original labels count: {LabelDataFrame.value_counts()}")
print(f"Integer labels count: {intDataFrame.value_counts()}")

images = (dataset['data'] / 255).astype(np.float32)
width = height = dataset['data'][0].shape[0]

Array Name: data
Array Shape:
(5200, 96, 96, 3)
Array Name: labels
Array Shape:
(5200,)
Original labels count: healthy      3199
unhealthy    2001
dtype: int64
Integer labels count: 0    3199
1    2001
dtype: int64


#Clean

In [5]:
#Clean wrong images

# Images from the dataset
X = images

# Create labels: 1 for 'unhealthy', 0 for 'healty'
y = intDataFrame

y = tfk.utils.to_categorical(y,len(np.unique(y)))

#Cleaning this shit out
target_value1 = images[506] #shrek

target_value2 = images[529] #trololo

# Create a boolean mask that checks for equality with the target image
mask = np.all(images == target_value1, axis=(1, 2, 3))

# Use the mask to filter the original array and remove matching instances
filtered_array = images[~mask]

matching_indices = np.where(mask)[0]
filtered_labels = y[~mask]

# Create a boolean mask that checks for equality with the target image
mask2 = np.all(filtered_array == target_value2, axis=(1, 2, 3))

# Use the mask to filter the original array and remove matching instances
filtered_array = filtered_array[~mask2]
filtered_labels = filtered_labels[~mask2]

print("Original array shape:", images.shape)
print("Filtered array shape:", filtered_array.shape)
print("Filtered labels shape:", filtered_labels.shape)

X = filtered_array
y = filtered_labels

#End  of Cleaning this shit out

Original array shape: (5200, 96, 96, 3)
Filtered array shape: (5004, 96, 96, 3)
Filtered labels shape: (5004, 2)


**Save cleaned**

In [6]:
np.save("X_cleaned.npy", X)
np.save("y_cleaned.npy", y)

#Balance

In [7]:
#Oversample using data augmentation the minor class


# Separate the dataset into majority and minority classes
majority_class_indices = np.where(y[:, 0] == 1)[0]
minority_class_indices = np.where(y[:, 1] == 1)[0]

# Calculate the size of the minority class
minority_size = len(minority_class_indices)

# Calculate the size of the majority class
majority_size = len(majority_class_indices)

# Calculate the number of additional samples to generate for the minority class
additional_minority_samples = majority_size - minority_size

# Data augmentation options for the oversampled minority class
datagen_minority = ImageDataGenerator(
    rotation_range=75,
    zoom_range=0.25,
    horizontal_flip=True
)

# Apply data augmentation to the minority class to reach the desired count
X_oversampled_minority_augmented = []
y_oversampled_minority_augmented = []
for index in minority_class_indices:
    x = X[index]
    X_oversampled_minority_augmented.append(x)  # Add the original sample
    y_oversampled_minority_augmented.append(y[index])  # Add the original label

# Randomly select and duplicate some augmented samples to reach the desired count
while additional_minority_samples > 0:
    index_to_duplicate = np.random.choice(minority_class_indices)
    x = X[index_to_duplicate]
    X_oversampled_minority_augmented.append(datagen_minority.random_transform(x))
    y_oversampled_minority_augmented.append(y[index_to_duplicate])
    additional_minority_samples -= 1

X_oversampled_minority_augmented = np.array(X_oversampled_minority_augmented)
y_oversampled_minority_augmented = np.array(y_oversampled_minority_augmented)

# Combine the oversampled and augmented minority class with the majority class
X_balanced = np.concatenate([X_oversampled_minority_augmented, X[majority_class_indices]])
y_balanced = np.concatenate([y_oversampled_minority_augmented, y[majority_class_indices]])

# Shuffle the balanced dataset
X_balanced, y_balanced = shuffle(X_balanced, y_balanced, random_state=seed)

X = X_balanced
y = y_balanced

**Save balanced**

In [8]:
X.shape

(6202, 96, 96, 3)

In [9]:
y.shape

(6202, 2)

In [10]:
np.save("X_balanced.npy",X)
np.save("y_balanced.npy",y)